# Topic WSD

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from os.path import join
import json
import numpy as np
from gensim import models, corpora
from LingPlus.wsd import TopicWSD
from LingPlus import wsd
import matplotlib.pyplot as plt

c:\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


http://127.0.0.1:8090/nlp/parse not responding


# 主題模型和詞義消岐

這份專案嘗試以主題模型（topic model），計算一個待消岐詞的「脈絡詞」，與一個詞義的CWN例句中的脈絡詞，兩者之間的詞彙連結度。並用該連結度（一個機率值）分辨待消岐詞是哪一個CWN的詞義（sense）。實作過程分為以下幾個階段
1. 以平衡語料庫訓練一個主題模型（主題數200）。
2. 把每一個CWN裡面的例句都當成是topic model裡的一篇「文章」，推測出其主題分配。
3. 選出每個例句中的目標詞（即使用該例句的詞彙）的鄰近3個詞，這些詞為該目標詞的「脈絡詞」。
4. 比對待消岐詞的脈絡詞，和每個詞義的例句中的脈絡詞，兩者間在主題模型中的詞彙連結度（word association）。
5. 待消岐詞的詞義分配，即是從和每個例句的詞彙連結度計算出來的詞義機率，在常規化（normalizae）後的數值。

### See derivation of probability distribution of sense [here](https://www.overleaf.com/read/gncxhcbcbmys)

# 一些測試數據

在CWN詞義中，我們挑選第2句例句作為測試句，其他例句為「參照句」。所以，若詞義無例句或只有1個例句的詞義，即不納入測試的例句。只有1個詞義的詞彙，雖然不需要詞義消岐，但仍納入測試範圍。
另外，為了描述TopicWSD所預測的詞義，是否比隨機預測的詞義更「接近」（亦即雖然不是它預測的詞義，但已經有較高機率是），
所以也計算該正確詞義在預測詞義中的排序。如果詞義消岐愈正確，詞義排序的值應該愈小（排序0即代表預測詞義也是正確的詞義）。

以下結果可詮釋為，如果待消岐句和例句的脈絡詞非常（甚至完全）相似，則TopicWSD有74%的機會可以預測到正確詞義，且即便預測錯誤，正確詞義平均而言也是第二高的預測詞義（詞義排序為0.81）。相對地，若待消岐句的脈絡詞是例句中從未出現的，則TopicWSD只有略高於機率水準的36%可以正確預測詞義。

## statistics on seen items
Total trial count: 2991 (sample 1000 lemmas)  
n_random_correct: 893, score: 0.30, mean-rank: 3.55  
n_twsd_correct: 2223, score: 0.74, mean-rank: 0.81  

## statistics on new items
Total trial count: 24098  

### Complete linkage
n_random_correct: 7241, score: 0.30, mean-rank: 4.12  
n_twsd_correct: 8725, score: 0.36, mean-rank: 3.33  

### Single linkage
n_random_correct: 7279, score: 0.30, mean-rank: 4.12  
n_twsd_correct: 8697, score: 0.36, mean-rank: 3.33  

In [5]:
topic_heads = []
with open("tm_topics.txt", "r", encoding="UTF-8") as fin:
    for ln in fin.readlines():
        words = ln.split(":")[1][2:-2]        
        words = [w.replace("'", "").strip() for w in words.split(',')]
        topic_heads.append(words)

In [6]:
DATA_PATH = r"E:\Kyle\LingNLP\week8_data"
model_path = join(DATA_PATH, "model/asbc5_200_gensim_pass20.model")
tm = models.LdaMulticore.load(model_path)
tm.gamma_threshold = 1e-5
dictionary = corpora.Dictionary.load(join(DATA_PATH, "model/gensim_asbc.dict"))

CWN_EXAMPLE_PATH = join(DATA_PATH, "cwn_example.json")
with open(CWN_EXAMPLE_PATH, "r", encoding="UTF-8") as fin:
    cwn = json.load(fin)

In [7]:
wsd.preproc.set_Oceanus_Endpoint("http://140.112.147.120:8090/nlp/parse")

In [8]:
twsd = TopicWSD(cwn, tm, dictionary)
twsd.sense_alpha = 1
twsd.HELDOUT_IDX = -1
twsd.linkage = TopicWSD.Linkage.Single

# An explaining example

In [9]:
def disambiguate(intext):
    token_list = wsd.preproc.segment(intext)
    print("/".join([x[0] for x in token_list]))

    for tok_i, tok in enumerate(token_list):
        wsd_data = wsd.preproc.preprocess(token_list, tok_i)
        lemma = tok[0]
        if twsd.has_lemma(lemma):
            print("--- %s ---" % lemma)
            psense = twsd.word_sense_disambiguate(wsd_data[1], wsd_data[0], lemma)            
            
            for sense_i in np.argsort([-x[1] for x in psense])[:5]:
                sense_x = psense[sense_i]
                print("[%s] %.4f: %s" % (sense_x[0], sense_x[1], twsd.get_sense_def(lemma, sense_x[0])))
            
    

In [10]:
lem = "等"
ref_list = twsd.get_disambiguate_refdata(lem)
ref_data = ref_list[0][1][0]
ambig_data = wsd.preproc.preprocess(wsd.preproc.segment("我們等好久"), 1)
print("n_sense of 等: ", len(ref_list))
print(ref_list[0][1][0])
print(ambig_data)

ambig_theta, ambig_word = twsd.get_anchor_topics(ambig_data[1], ambig_data[0])
print("Ambig context: ", ambig_word)
# compute compatability score
import LingPlus.wsd.topic_utils as tu
for senseid, examples in ref_list:
    print(senseid)
    for ref_data in examples:                
        ref_theta, ref_word = twsd.get_anchor_topics(ref_data[1], ref_data[0])        
        print("Reference context:", ref_word)
        if ref_word and ambig_word:
            comp_score = tu.get_word_assoc(ambig_word, ref_word, ref_theta, 
                    twsd.model, twsd.dictionary)
            print("Association prob: ", comp_score[0])

disambiguate("我們等好久")            

n_sense of 等:  8
(13, ['守時', '是', '很', '重要', '的', '事', '。', '不', '能', '遲到', '，', '讓', '大家', '等', '你', '一', '個', '人', '。'])
(1, ['我們', '等', '好久'])
Ambig context:  ['我們', '好久']
04001101
Reference context: ['，', '讓', '大家', '你', '一', '個']
Association prob:  0.0199639918751
Reference context: ['一', '扔', '，', '看', '電視', '。']
Association prob:  0.0065223789267
Reference context: ['兩', '短', '」', '掛號', '、', '等']
Association prob:  0.000274923395592
04001102
Reference context: ['分', '工作', '，', '錢', '存', '夠']
Association prob:  0.00839079840987
Reference context: ['他', '出去', '，', '天', '黑', '了']
Association prob:  0.00651695967337
Reference context: ['產下', '小熊', '，', '小熊', '滿', '一']
Association prob:  0.00339941547101
Reference context: ['兩', '年', '，', '念', '完', '了']
Association prob:  0.0137128577981
Reference context: ['一旁', '休息', '，', '天黑', '後', '，']
Association prob:  0.00652270265882
04001103
Reference context: ['島', '的', '面積', '。']
Association prob:  0.000493413707577
Reference context: ['另'

# 討論

1. TopicWSD完全仰賴主題模型來建立例句和待消岐句之間的關係，所以主題模型的參數，
例如主題數，詞彙前處理（尤其是去掉高低頻詞的程序），模型的hyperparameters，都可能影響最後的WSD結果。
只是在CWN的資料裡，詞彙的詞義相當多，而每個詞義又只有3個左右的例句，可能較難以cross-validation的方式調校參數。

2. 主題模型也是一種語義表徵的向量空間模型，而且其輸入材料基本上也是詞袋（bag of word）資料，詞的順序、詞類、句法等訊息是完全忽略的。
有兩種可能的想法可以加入更多的語言訊息，例如（1）先將句子作句法剖析，用句法剖析來決定脈絡詞。例如透過dependency grammar，
發現動詞「等」的主語或賓語，並用這兩個詞當作脈絡詞，或許能比現有作法更能含括到相關訊息；（2）不要用主題模型，而改用其他向量空間模型，
並同時以詞彙共現與句法關係當作輸入，估計詞彙向量，並用該向量表徵計算待消岐句與例句的相似度；（3）用CWN本身的訊息加上句法結構作詞彙消岐，
但可能會對資料的覆蓋率有其他的要求。

3. TopicWSD在計算待消岐句與例句的詞彙關連度後，目前可以用Complete或Single linkage決定最後的詞義。沿襲階層群句法（hierarchical clustering）
的定義，Complete linkage是以一個詞義中所有的例句，和待消岐句的詞彙關連度平均，當作該詞義最後與待消岐詞的機率；相對地，Single linkage是以一個詞義中
機率最剛的那個例句，當作該詞義與待消岐句的機率。從實際應用的角度而言，一個詞義在CWN裡的例句可能各自是不同的脈絡，事實上只要有任一個例句
和待消岐句的脈絡接近，它很可能就是屬於該詞義，而不需要真正符合所有該詞義中的所有例句的脈絡。所以在實用上，Single linkage可能是較實用的。

4. 雖然在測試未知句子時，用uniform prior的正確率只有0.36。但實際上，並不是所有CWN列出來的詞義在真實語料中的出現機率都一致。
甚至在CWN裡，排在前面的詞義「或許」反應其常用程度。故以詞義出現順序來調整先驗機率，可增加此方法在真實語料中的實用性。

# WSD Examples

In [11]:
disambiguate("老師說不能夠打同學")

老師/說/不/能夠/打/同學
--- 老師 ---
[06667701] 1.0000: 具有教學能力或身份的人。
--- 說 ---
[05212412] 0.2206: 無義，表結束或停頓的語氣。
[05212411] 0.2206: 無義，表後述單位為句賓。
[05212406] 0.2198: 評價後述對象。
[05212405] 0.2038: 指涉後述對象。
[05212409] 0.0814: 說明特定事件的原因、理由使聽話者明白。
--- 不 ---
[05010901] 0.8096: 表疑問的語氣，置於句末。
[04002001] 0.1904: 表否定，用在動詞、狀態形容詞與副詞前。
[05010801] 0.0000: 不丹。
--- 能夠 ---
[05196302] 0.9988: 表具備達到後述事件成立的條件。
[05196301] 0.0012: 形容具備完成特定事件能力的。
[05196303] 0.0000: 表具有後述功能。
--- 打 ---
[052291b3] 0.0410: 發聲時，使舌頭快速振動。
[05229124] 0.0410: 計算節拍以控制音樂進行的速度。
[05229184] 0.0410: 拿比較熟悉的事物來描述、解釋另一事件。
[05229178] 0.0410: 搭乘有座位的交通工具。
[05229111] 0.0410: 比喻建立基礎。
--- 同學 ---
[06612801] 0.2987: 一起學習的人。
[06612805] 0.2984: 在同一團體中一起進行特定事件的人。
[06612803] 0.2976: 對學習者的稱呼。
[06612804] 0.0576: 對不熟識的可能是在學年齡的對象的稱呼。
[06612802] 0.0477: 一起學習。


In [12]:
disambiguate("許多遊客為了取景拍照直接踏進稻田裡")

許多/遊客/為了/取景/拍照/直接/踏進/稻田/裡
--- 許多 ---
[05172701] 0.9879: 數量大或種類多。
[05172702] 0.0121: 表前述特質的程度高。
--- 遊客 ---
[06604001] 1.0000: 前往特定地點休閒或娛樂的人。
--- 為了 ---
[07025201] 1.0000: 引介事件原因。
--- 直接 ---
[06772503] 0.8487: 表不避諱。
[06772504] 0.1513: 形容表達方式直接的。
[06772505] 0.0000: 形容比喻不必透過媒介就能產生關連的。
[06772501] 0.0000: 表不改變方向向後述目的地運動。
[06772506] 0.0000: 表不必透過其他媒介來做後述事件。
--- 裡 ---
[04086801] 0.6270: 事物內部不屬於表面的範圍。
[04086802] 0.3683: 在前述對象所包含的空間範圍。
[04086804] 0.0045: 在前述的機構或領域範圍。
[04086803] 0.0001: 在前述的時間範圍內。


In [13]:
disambiguate("思考必定先有一個搞清楚的問題，然後多方去找解答，比較後得出一個可以說服自己的答案。")

思考/必定/先/有/一/個/搞/清楚/的/問題/，/然後/多方/去/找/解答/，/比較/後/得出/一/個/可以/說服/自己/的/答案/。
--- 必定 ---
[05097001] 1.0000: 表對推論的肯定，沒有例外。
--- 先 ---
[06532401] 0.6292: 表時間或順序在前的。
[07011801] 0.3053: 姓。
[06532402] 0.0394: 處理時間或順序排在較前的特定對象。
[06532404] 0.0218: 進行後述事件的時間比其他特定對象早。
[06532405] 0.0040: 特定民族或宗族的上代。
--- 有 ---
[04013807] 0.8084: 表肯定事件的發生。
[04013802] 0.1177: 斷定特定對象有後述性質。
[04013803] 0.0570: 後述對象存在。
[06036901] 0.0083: 給予對方的回應。
[04013805] 0.0066: 達到後述程度。
--- 一 ---
[06727304] 0.3609: 表完成後述動作後，緊接著發生相關事件結果。
[05224201] 0.2834: 數目字。整數，最小的正整數。
[06727401] 0.2153: 姓。
[05224202] 0.0639: 序數。順序排在第一。
[06727301] 0.0388: 事物的全體。
--- 個 ---
[06533601] 0.4308: 人的身高。
[07011204] 0.3612: 計算特定對象大約的數量的單位。
[07011206] 0.1068: 表加強語氣，強調前述動作。
[07011207] 0.0556: 表以輕鬆的態度提出進行特定事件的要求。
[07011201] 0.0251: 計算能獨立成件的具體對象的單位。
--- 清楚 ---
[06664803] 0.8312: 清楚地知道。
[06664801] 0.0889: 形容感官上明顯容易辨識。
[06664802] 0.0799: 形容認知的內容容易明確。
--- 的 ---
[07023407] 0.6161: 表肯定或加強的語氣。
[07023405] 0.3745: 代指後述省略的特定對象。
[07023406] 0.0038: 表在前述情況下。
[09003002] 0.0026: 表對推論的

In [14]:
disambiguate("網友認為其實麵包店土司更貴，還有人抗議為什麼原物料一直上漲，薪水卻沒漲。")

網友/認為/其實/麵包店/土司/更貴/，/還有/人/抗議/為什麼/原物料/一直/上漲/，/薪水/卻/沒/漲/。
--- 認為 ---
[05201101] 1.0000: 引述或陳述前述對象的認知。
--- 其實 ---
[05100501] 1.0000: 表事件的實際情況。
--- 還有 ---
[04000401] 1.0000: 連接語意相似的詞組，表另外有的意思。
--- 人 ---
[05231109] 0.4607: 符合特定需求、從事特定工作的人。
[05231107] 0.4129: 人的生理狀態。
[05231111] 0.0855: 前述對象的跟隨者。
[05231110] 0.0276: 人的本質特性。
[05231105] 0.0128: 自己以外的人。
--- 為什麼 ---
[03044902] 1.0000: 事件的原因或理由，表疑問之意。
[03044901] 0.0000: 表對事件的疑問，詢問原因。
--- 一直 ---
[06640105] 1.0000: 表事件狀態由前述描述發展到後述描述。
[06640104] 0.0000: 表在後述時點前事件沒有改變。
[06640101] 0.0000: 表不改變方向向後述目的地運動。
[06640102] 0.0000: 表事件持續不間斷。
[06640103] 0.0000: 表狀態從過去持續到參考時點，強調其狀態沒有改變過。
--- 卻 ---
[06614301] 1.0000: 表轉折。
--- 沒 ---
[03028101] 0.8250: 不擁有後述對象。
[03028103] 0.0626: 未達一定的數量。
[03028106] 0.0306: 表示疑問，置於句末。
[03028104] 0.0280: 否定事件未達後述的程度。
[03028102] 0.0274: 後述對象不存在。
--- 漲 ---
[08024601] 0.9537: 使特定對象價格提高。
[08024501] 0.0289: 形容水位上升。
[06567702] 0.0149: 形容頭部充血。
[06567701] 0.0025: 形容前述對象吸收水份後體積變大。
[08024502] 0.0000: 形容比喻價格上升。
